去除小于2000的数据

In [ ]:
import pandas as pd
def filter_and_save_excel():
    try:
        # 获取输入文件路径
        input_file = input("请输入要读取的 Excel 文件路径：")
        # 获取输出文件路径
        output_file = input("请输入要保存的 Excel 文件路径：")
        # 获取要去掉的字段名，用空格分隔
        fields_to_remove = input("请输入要去掉的字段名，用空格分隔（例如：字段1 字段2）：").split()

        # 读取 Excel 文件，会返回一个字典，键为 sheet 名，值为对应的 DataFrame
        excel_file = pd.ExcelFile(input_file)
        sheet_names = excel_file.sheet_names

        # 创建一个 ExcelWriter 对象，用于将不同 sheet 的筛选结果写入同一个文件
        with pd.ExcelWriter(output_file) as writer:
            for sheet_name in sheet_names:
                # 获取当前 sheet 的数据
                df = excel_file.parse(sheet_name)

                # 去掉指定的字段
                df = df.drop(columns=[col for col in fields_to_remove if col in df.columns])

                # 假设价格列名为 '价格'，你可以根据实际情况修改
                if '价格' in df.columns:
                    # 筛选价格大于 2000 元的数据
                    filtered_df = df[df['价格'] > 2000]

                    # 将筛选后的数据保存到新文件的对应 sheet 中
                    filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"工作表 '{sheet_name}' 筛选后的数据已保存到 {output_file} 的同名工作表中。")
                else:
                    print(f"工作表 '{sheet_name}' 中没有找到 '价格' 列，请检查列名。")
    except FileNotFoundError:
        print(f"未找到文件 {input_file}，请检查文件路径。")
    except Exception as e:
        print(f"发生错误: {e}")

# 调用函数
filter_and_save_excel()


# 商品链接 评论链接 店铺链接 封面图片链接 页码	当前时间	页面网址 jd
# 店铺链接 图片地址	商品链接	商品id	当前页面网址	当前时间	页码 tb 产品价格



In [ ]:
import pandas as pd # type: ignore

# 读取 Excel 文件
excel_file = pd.ExcelFile('京东评论.xlsx')

# 获取所有表名
sheet_names = excel_file.sheet_names

# 遍历每个 sheet
for sheet_name in sheet_names:
    print(f'正在分析表 {sheet_name}...')
    # 获取当前 sheet 的数据
    data_jingdong = excel_file.parse(sheet_name)

    # 按月统计京东店铺的销售数量；按商品属性统计商品的销售情况；按评分统计评价人数并计算商品在京东的评分
    level = data_jingdong['级别']
    star = data_jingdong['评价星级']
    eva_time = data_jingdong['时间']
    style = data_jingdong['商品属性']

    # 将时间转换为日期时间类型并提取月份
    data_jingdong['时间'] = pd.to_datetime(data_jingdong['时间'])
    data_jingdong['月份'] = data_jingdong['时间'].dt.month

    # 提取评分
    data_jingdong['score'] = [int(i[-1]) for i in data_jingdong['评价星级']]

    # 按照月份和评分分组统计销售情况（用评论数量近似代表销售数量）
    monthly_score_sales = data_jingdong.groupby(['月份', 'score'])['会员'].count().unstack(fill_value=0)

    # 各月份的用户评论条数
    month_counts = data_jingdong['月份'].value_counts()

    # 各属性购买数量
    group_data = data_jingdong.groupby('商品属性')['会员'].count()

    # 提取最高的商品属性和销售数量
    pname = group_data.idxmax()
    pnum = group_data.max()

    # 按照会员和非会员分别统计人数
    plus_num = data_jingdong.groupby('级别')['会员'].count()[0]

    # 计算这件商品的平均评分
    ave_score = data_jingdong['score'].mean()

    # 保存结果到 txt 文件
    txt_filename = f'{sheet_name}.txt'
    with open(txt_filename, 'w', encoding='utf-8') as f:
        f.write(f'表 {sheet_name} 的分析结果：\n')
        f.write('-' * 50 + '\n')
        f.write('各月份的用户评论条数：\n')
        f.write(month_counts.to_csv(sep='\t', na_rep='nan') + '\n')
        f.write('-' * 50 + '\n')
        f.write('各属性购买数量：\n')
        f.write(group_data.to_csv(sep='\t', na_rep='nan') + '\n')
        f.write('-' * 50 + '\n')
        f.write(f'销量最高的商品是<{pname}>,一共销售了{pnum}份\n')
        f.write('-' * 50 + '\n')
        f.write(f'PLUS会员占比{plus_num/len(data_jingdong)*100:.2f}%\n')
        f.write('-' * 50 + '\n')
        f.write(f'这件商品的平均评分是{ave_score:.2f}\n')
        f.write('-' * 50 + '\n')
        f.write('按照评分统计的每个月销售情况（用评论数量近似代表销售数量）：\n')
        f.write(monthly_score_sales.to_csv(sep='\t', na_rep='nan') + '\n')

    print(f'表 {sheet_name} 的分析结果已保存到 {txt_filename}')
    print('-' * 50)

In [ ]:
import pandas as pd

# 读取 Excel 文件
excel_file = pd.ExcelFile('jdGoogle111.xlsx')

# 创建一个 ExcelWriter 对象，用于将处理后的数据写回文件
with pd.ExcelWriter('jdGoogle111.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # 遍历每个 sheet
    for sheet_name in excel_file.sheet_names:
        df = excel_file.parse(sheet_name)
        # 将“万”替换为“0000”
        df['评价人数'] = df['评价人数'].str.replace('万', '0000')
        # 去除非数字字符，只保留数字
        df['评价人数'] = df['评价人数'].str.extract('(\d+)', expand=False)
        # 将数据类型转换为数值型（可选，根据实际需求）
        df['评价人数'] = pd.to_numeric(df['评价人数'], errors='coerce')
        # 处理标签列，空格缺省的为0，反之为1
        df['标签'] = df['标签'].notnull().astype(int)
        # 处理是否广告列
        df['是否广告'] = df['是否广告'].apply(lambda x: 1 if x == '广告' else 0)
        # 将处理后的数据写回到对应的 sheet 中
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
import pandas as pd

# 读取 Excel 文件
excel_file = pd.ExcelFile('tbGoogle 222.xlsx')

# 创建一个 ExcelWriter 对象，用于将处理后的数据写回文件
with pd.ExcelWriter('tbGoogle 222.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # 遍历每个 sheet
    for sheet_name in excel_file.sheet_names:
        df = excel_file.parse(sheet_name)
        df['付款人数'] = df['付款人数'].str.replace('万', '0000')
        # 去除非数字字符，只保留数字
        df['付款人数'] = df['付款人数'].str.extract('(\d+)', expand=False)
        # 将数据类型转换为数值型（可选，根据实际需求）
        df['付款人数'] = pd.to_numeric(df['付款人数'], errors='coerce')
        # 将处理后的数据写回到对应的 sheet 中
        df.to_excel(writer, sheet_name=sheet_name, index=False)    
        print(f'{sheet_name}已完成')         

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import pearsonr
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression



# 读取 Excel 文件
excel_file = pd.ExcelFile('jdGoogle111.xlsx')

# 遍历每个工作表
for sheet_name in excel_file.sheet_names:
    # 读取当前工作表的数据
    df = excel_file.parse(sheet_name)
    # 筛选评价人数大于 10 的数据
    df = df[df['评价人数'] > 10]

    # 1.描述性统计
    price_stats = df['价格'].describe()
    print(f'_______________{sheet_name}_______________')
    print(price_stats)



    # 设置图片分辨率
    plt.rcParams['figure.dpi'] = 300
    # 获取价格的最大值和最小值
    max_price = df['价格'].max()
    min_price = df['价格'].min()
    # 绘制价格的频数直方图
    n, bins, patches = plt.hist(df['价格'], bins=8, edgecolor='black', density=True)
    # 拟合正态分布
    mu, std = norm.fit(df['价格'])
    xmin = min_price
    xmax = max_price
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    # 设置 x 轴范围
    plt.xlim(xmin, xmax)
    # 绘制正态分布曲线，修改颜色为红色
    plt.plot(x, p, color='red', linewidth=2, label='正态分布')
    # 解决中文显示问题
    plt.rcParams['font.sans-serif'] = ['SimHei']
    # 解决负号显示问题
    plt.rcParams['axes.unicode_minus'] = False
    # 设置图表标题和坐标轴标签
    plt.title(sheet_name + '价格频数直方图')
    plt.xlabel('价格')
    plt.ylabel('频数')
    plt.legend()
    # 显示直方图
    plt.show()

    # 创建新的图形用于绘制箱线图
    plt.figure()

    # 绘制带颜色的箱线图
    boxprops = dict(linestyle='-', linewidth=2, color='blue')
    medianprops = dict(linestyle='-', linewidth=2, color='red')
    whiskerprops = dict(linestyle='--', linewidth=1.5, color='green')
    capprops = dict(linestyle='-', linewidth=1.5, color='orange')
    flierprops = dict(marker='o', markerfacecolor='yellow', markersize=8, markeredgecolor='black')

    boxplot = plt.boxplot(df['价格'], boxprops=boxprops, medianprops=medianprops,
                          whiskerprops=whiskerprops, capprops=capprops, flierprops=flierprops)

    # 获取箱线图的关键数据
    medians = [item.get_ydata()[0] for item in boxplot['medians']]
    q1 = [item.get_ydata()[0] for item in boxplot['boxes']]
    q3 = [item.get_ydata()[2] for item in boxplot['boxes']]
    whiskers_min = [item.get_ydata()[1] for item in boxplot['whiskers'][::2]]
    whiskers_max = [item.get_ydata()[1] for item in boxplot['whiskers'][1::2]]

    # 标注箱线图的关键数据
    plt.text(1.1, medians[0], f"中位数: {medians[0]:.2f}", verticalalignment='center')
    plt.text(1.1, q1[0], f"Q1: {q1[0]:.2f}", verticalalignment='center')
    plt.text(1.1, q3[0], f"Q3: {q3[0]:.2f}", verticalalignment='center')
    plt.text(1.1, whiskers_min[0], f"最小值: {whiskers_min[0]:.2f}", verticalalignment='center')
    plt.text(1.1, whiskers_max[0], f"最大值: {whiskers_max[0]:.2f}", verticalalignment='center')

    # 创建自定义图例
    median_line = plt.Line2D([0], [0], color='red', linewidth=2)
    box_line = plt.Line2D([0], [0], color='blue', linewidth=2)
    whisker_line = plt.Line2D([0], [0], color='green', linestyle='--')
    cap_line = plt.Line2D([0], [0], color='orange', linewidth=1.5)
    flier_marker = plt.Line2D([0], [0], marker='o', markerfacecolor='yellow', markersize=8, markeredgecolor='black', linestyle='None')
    plt.legend([median_line, box_line, whisker_line, cap_line, flier_marker], ['中位数', '箱体', '须线', '端点线', '异常值'])

    # 设置箱线图的标题和 y 轴标签
    plt.title(sheet_name + '价格箱线图')
    plt.ylabel('价格')
    # 显示箱线图
    plt.show()

 


    #相关性分析
    # 1. 绘制散点图
    plt.figure()
    plt.scatter(df['价格'], df['评价人数'])
    plt.xlabel('价格')
    plt.ylabel('销量')
    plt.title(sheet_name+'价格与销量的散点图')
    plt.show()
    # 2. 计算皮尔逊相关系数和 p 值
    correlation, p_value = pearsonr(df['价格'], df['评价人数'])
    print(f'---------------{sheet_name}---------------')
    print(f'皮尔逊相关系数: {correlation}')
    print(f'p 值: {p_value}')

    # 3. 线性回归分析
    # 自变量
    X = sm.add_constant(df['价格'])
    # 因变量
    y = df['评价人数']
    # 创建线性回归模型
    model = sm.OLS(y, X)
    # 拟合模型
    results = model.fit()
    # 打印回归结果
    print(results.summary())


    #聚类分析
    # 1. 进行K - means聚类分析
    # 选择要聚类的特征，这里仅使用价格
    X = df[['价格','评价人数']]
    # 假设我们将价格分为3个区间，即聚类为3类
    kmeans = KMeans(n_clusters=6, random_state=42)
    kmeans.fit(X)
    # 获取每个数据点的聚类标签
    df['聚类标签'] = kmeans.labels_

    # 2. 确定价格区间
    # 按聚类标签分组，找出每个聚类中价格的最小值和最大值
    cluster_price_ranges = df.groupby('聚类标签')['价格'].agg(['min', 'max'])
    print("各聚类的价格区间：")
    print(cluster_price_ranges)


    # 3. 可视化聚类结果
    plt.scatter(df['价格'], df['评价人数'], c=df['聚类标签'], cmap='viridis')
    # 绘制聚类中心
    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x', s=200)
    plt.xlabel('价格')
    plt.ylabel('销量')
    plt.title(sheet_name+' K - means 聚类结果（包含价格和销量特征）')
    plt.colorbar(label='聚类标签')
    plt.show()

    # 4. 简单的分段回归示例
    from sklearn.linear_model import LinearRegression

    # 根据聚类标签将数据分为不同的段
    for cluster_label in df['聚类标签'].unique():
        subset = df[df['聚类标签'] == cluster_label]
        X_subset = subset[['价格']]
        y_subset = subset['评价人数']
        # 创建线性回归模型
        model = LinearRegression()
        model.fit(X_subset, y_subset)
        # 打印每个段的回归系数和截距
        print(f"聚类标签 {cluster_label} 的回归系数: {model.coef_[0]}, 截距: {model.intercept_}")

    
    # 特征变量直方图
    # 创建一个包含两个子图的图形
    # plt.figure()
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # 绘制标签列的直方图，设置颜色为蓝色，透明度为 0.7
    df['标签'].hist(ax=axes[0], bins=[-0.5, 0.5, 1.5], edgecolor='black', color='blue', alpha=0.7)
    axes[0].set_title(sheet_name+'标签列直方图')
    axes[0].set_xlabel('标签值')
    axes[0].set_ylabel('频数')
    axes[0].set_xticks([0, 1])
    # 去除网格线
    axes[0].grid(False)

    # 绘制是否广告列的直方图，设置颜色为绿色，透明度为 0.6
    df['是否广告'].hist(ax=axes[1], bins=[-0.5, 0.5, 1.5], edgecolor='black', color='green', alpha=0.6)
    axes[1].set_title(sheet_name+'是否广告列直方图')
    axes[1].set_xlabel('是否广告值')
    axes[1].set_ylabel('频数')
    axes[1].set_xticks([0, 1])
    # 去除网格线
    axes[1].grid(False)

    # 调整子图之间的间距
    plt.tight_layout()

    # 显示图形
    plt.show()



    #价格分布图

    df = df[df['价格']>3000]
    # 定义价格分段
    bins = [ 3000, 5000, 7000, 9000, 11000,13000, float('inf')]
    labels = ['3000-5000', '5001-7000', '7001-9000', '9001-11000', '11001-13000', '13001+']

    # 对价格进行分段
    df['价格分段'] = pd.cut(df['价格'], bins=bins, labels=labels)

    # 统计每个价格分段的数量
    segment_counts = df['价格分段'].value_counts()

    # 绘制饼图
    # plt.figure(figsize=(8, 8))
    # plt.pie(segment_counts, labels=segment_counts.index, autopct='%1.1f%%', startangle=140)
    # plt.axis('equal')  # 保证饼图是圆形
    # plt.title(sheet_name+'价格分段分布饼图')
    # plt.show()
    

    # 绘制饼图
    fig, ax = plt.subplots(figsize=(8, 8))
    # 接收 ax.pie() 返回的三个值
    wedges, texts, autotexts = ax.pie(segment_counts, labels=None, autopct='%1.1f%%', startangle=140)

    # 调整图例
    ax.legend(wedges, labels, title="价格区间", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    # 调整图表标题
    ax.set_title(sheet_name+'价格分段分布饼图', fontsize=16)

    # 确保饼图为圆形
    ax.axis('equal')

    # 显示图表
    plt.show()






In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 读取 Excel 文件
excel_file = pd.ExcelFile('jdGoogle111.xlsx')

# 遍历每个工作表
for sheet_name in excel_file.sheet_names:
    # 读取当前工作表的数据
    df = excel_file.parse(sheet_name)
    # 筛选评价人数大于 10 的数据
    df = df[df['评价人数'] > 10]


    # 2. 数据探索：计算不同组合下的平均销量
    grouped_data = df.groupby(['是否广告', '标签'])['评价人数'].mean().reset_index()

    # 3. 可视化分析

    # 设置图片清晰度
    plt.rcParams['figure.dpi'] = 300

    # 箱线图展示是否广告与销量的关系
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.boxplot(x='是否广告', y='评价人数', data=df)
    plt.title(sheet_name+'Relationship between Advertisement and sale')
    plt.xlabel('Is Advertised (0: No, 1: Yes)')
    plt.ylabel('sale')

    # 箱线图展示标签与销量的关系
    plt.subplot(1, 2, 2)
    sns.boxplot(x='标签', y='评价人数', data=df)
    plt.title(sheet_name+'Relationship between Label and sale')
    plt.xlabel('Has Label (0: No, 1: Yes)')
    plt.ylabel('sale')

    plt.tight_layout()
    plt.show()

    # 柱状图展示不同组合下的平均销量
    plt.figure(figsize=(8, 6))
    sns.barplot(x='是否广告', y='评价人数', hue='标签', data=grouped_data)
    plt.title(sheet_name+'Average sale by Advertisement and Label Combinations')
    plt.xlabel('Is Advertised (0: No, 1: Yes)')
    plt.ylabel('Average sale')
    plt.legend(title='Has Label', labels=['No', 'Yes'])
    plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 读取 Excel 文件
excel_file = pd.ExcelFile('jdGoogle111.xlsx')

# 用于存储每个工作表处理后的数据
all_dfs = []

# 遍历每个工作表
for sheet_name in excel_file.sheet_names:
    # 读取当前工作表的数据
    df = excel_file.parse(sheet_name)
    # 筛选评价人数大于 10 的数据
    df = df[df['评价人数'] > 10]
    all_dfs.append(df)

# 将所有工作表的数据合并
combined_df = pd.concat(all_dfs, ignore_index=True)

# 选取需要分析相关性的列
selected_columns = ['评价人数', '标签', '是否广告']
correlation_df = combined_df[selected_columns]

# 计算相关系数矩阵
corr_matrix = correlation_df.corr()

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 绘制热力图
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')


# 显示图表
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取 Excel 文件
excel_file = pd.ExcelFile('京东评论.xlsx')

# 获取所有表名
sheet_names = excel_file.sheet_names

# 遍历每个 sheet
for sheet_name in sheet_names:
    print(f'正在分析表 {sheet_name}...')
    # 获取当前 sheet 的数据
    data_jingdong = excel_file.parse(sheet_name)

    # 按月统计京东店铺的销售数量；按商品属性统计商品的销售情况；按评分统计评价人数并计算商品在京东的评分
    level = data_jingdong['级别']
    star = data_jingdong['评价星级']
    eva_time = data_jingdong['时间']
    style = data_jingdong['商品属性']

    # 各月份的用户评论条数
    data_jingdong['时间'] = pd.to_datetime(data_jingdong['时间'])
    data_jingdong['月份'] = data_jingdong['时间'].dt.month
    monthly_comments = data_jingdong['月份'].value_counts().sort_index()
    print(f'表 {sheet_name} 各月份的用户评论条数：')
    print(monthly_comments)


    # 绘制各月份用户评论条数折线图
    plt.figure(figsize=(10, 6))
    monthly_comments.plot(kind='line', marker='o')
    plt.title(f'表 {sheet_name} 各月份的用户评论条数')
    plt.xlabel('月份')
    plt.ylabel('评论条数')
    plt.xticks(rotation=0)
    plt.show()

    # 各属性购买数量
    group_data = data_jingdong.groupby('商品属性')['会员'].count()
    print(f'表 {sheet_name} 各属性购买数量：')
    print(group_data)

   

    # # 绘制各属性购买数量饼图
    # plt.figure(figsize=(8, 8))
    # group_data.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    # plt.title(f'表 {sheet_name} 各属性购买数量占比')
    # plt.ylabel('')  # 隐藏 y 轴标签
    # plt.show()

    # 绘制各属性购买数量饼图
    plt.figure(figsize=(8, 8))
    # 绘制饼图，设置 labeldistance 为 None 避免标签与图例冲突
    patches, texts, autotexts = plt.pie(group_data, autopct='%1.1f%%', startangle=90)    
    plt.title(f'表 {sheet_name} 各属性购买数量占比')
    plt.ylabel('')  # 隐藏 y 轴标签
    # 添加图例
    plt.legend(patches, group_data.index, loc="best")
    plt.show()


    # 绘制柱状图
    plt.figure(figsize=(10, 6))
    ax = group_data.plot(kind='bar')

    # 添加数据标签
    for p in ax.patches:
        ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    # 设置图表标题和坐标轴标签
    plt.title(f'表 {sheet_name} 各商品属性的购买数量')
    plt.xlabel('商品属性')
    plt.ylabel('购买数量')

    # 旋转 x 轴标签以避免重叠
    plt.xticks(rotation=45)

    # 显示图表
    plt.show()


    # 提取最高的商品属性和销售数量
    pname = group_data.idxmax()
    pnum = group_data.max()
    print(f'表 {sheet_name} 销量最高的商品是<{pname}>,一共销售了{pnum}份')

    # 按照会员和非会员分别统计人数
    plus_num = data_jingdong.groupby('级别')['会员'].count()[0]
    print(f'表 {sheet_name} PLUS会员占比{plus_num/len(data_jingdong)*100:.2f}%')

    # 取出 star5
    data_jingdong['score'] = [int(i[-1]) for i in data_jingdong['评价星级']]
    ave_score = data_jingdong['score'].mean()
    print(f'表 {sheet_name} 这件商品的平均评分是{ave_score:.2f}')

    print('-' * 50)

In [ ]:
# 词频统计，评论分析

import pandas as pd
import matplotlib.pyplot as plt
import jieba
import wordcloud as wc
from jieba import analyse #关键词提取模块
import numpy as np
from PIL import Image



# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取 Excel 文件
excel_file = pd.ExcelFile('京东评论.xlsx')

# 获取所有表名
sheet_names = excel_file.sheet_names

# 遍历每个 sheet
for sheet_name in sheet_names:
    print(f'正在分析表 {sheet_name}...')
    # 获取当前 sheet 的数据
    data_jingdong = excel_file.parse(sheet_name)
    comment = data_jingdong['评价内容']
    # 提取全部用户评价，并且组合在一个字符串evaluation当中
    evaluation = ''.join(comment)
    jieba.load_userdict('用户自定义词典.txt')#加载用户自定义词典
    # print('使用自定义词典后分词结构如下：')
    words = jieba.lcut(evaluation) #lcut 会将分词结果保存在一个列表里
    #过滤掉停用词
    # 1.加载停用词列表
    with open('停用词.txt', encoding = 'utf-8') as f:
        stop_words = [w.strip() for w in f]
    # 2.从分词结果中过滤停用词      
    words = [w for w in words if w not in stop_words]
    # print(words)
    # words转化成pandas的系列类型
    words = pd.array(words)
    word_fre = words.value_counts()
    # 从evaluation提取关键词
    # 提取评分的最高的前70个词，要求只选择名词
    keyWords = analyse.textrank(evaluation,topK=70,allowPOS=['n'])
    keyWords = [word for word in keyWords if word in words]
    # print(keyWords)
    # 从word_fre中提取出所有keyWords的词频
    key_fre = word_fre[keyWords]
    # print(key_fre)
    # 可视化结果--画图
    #导入画图所需模块，词云图
    # 1.条形图
    plt.bar(key_fre.index[:10],key_fre.values[:10])
    plt.title(f'{sheet_name}评论关键词词频图')
    plt.show()
    # plt.savefig('联想ThinkPadT14p词频图.png')
    # 2.词云图
    # （1）转化为字典
    key_d = dict(zip(key_fre.index,key_fre.values))
    # print(key_d)
    # （2）画图
    myfont = r'C:\Windows\Fonts\simhei.ttf'
    mymask = np.array(Image.open(f'logo\y7000p.png'))
    fig=wc.WordCloud(font_path=myfont,background_color='white',mask=mymask).fit_words(key_d)
    plt.imshow(fig)
    plt.title(f'{sheet_name}词云图')
    plt.axis("off")
    plt.show()





In [ ]:
#情感分析
from snownlp import SnowNLP
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取 Excel 文件
excel_file = pd.ExcelFile('京东评论.xlsx')

# 获取所有表名
sheet_names = excel_file.sheet_names

# 遍历每个 sheet
for sheet_name in sheet_names:
    print(f'正在分析表 {sheet_name}...')
    # 获取当前 sheet 的数据
    data_jingdong = excel_file.parse(sheet_name)
    comment = data_jingdong['评价内容']
    
    # Convert sentiment scores to labels
    score_l = []
    labels = []
    for text in comment:
        s = SnowNLP(text)
        score = s.sentiments
        score_l.append(score)
        # Convert scores to labels (negative: 0, neutral: 1, positive: 2)
        if score < 0.4:
            labels.append(0)  # Negative
        elif score < 0.6:
            labels.append(1)  # Neutral
        else:
            labels.append(2)  # Positive
    # 情感波动图
    plt.title(f'{sheet_name}情感波动图')
    plt.plot(score_l)
    plt.show()

    # Convert text to TF-IDF features
    vectorizer = TfidfVectorizer(max_features=1000)
    X = vectorizer.fit_transform(comment)

    # Train SVM classifier
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)

    # Predict and evaluate
    predictions = svm.predict(X_test)
    accuracy = (predictions == y_test).mean()

    # Count sentiment distributions
    sentiment_counts = pd.Series(labels).value_counts()

    # Visualize results
    plt.figure(figsize=(10, 6))
    plt.bar(['Negative', 'Neutral', 'Positive'], 
            [sentiment_counts.get(0, 0), sentiment_counts.get(1, 0), sentiment_counts.get(2, 0)])
    plt.title(f'{sheet_name}Sentiment Distribution of Reviews')
    plt.ylabel('Number of Reviews')
    plt.show()
    # plt.savefig('sentiment_distribution.png')
    ave_score_l = sum(score_l)/len(score_l)
    print(f"分类准确率: {accuracy:.2f}")
    print(f"正面评价数量: {sentiment_counts.get(2, 0)}")
    print(f"中性评价数量: {sentiment_counts.get(1, 0)}")
    print(f"负面评价数量: {sentiment_counts.get(0, 0)}")
    print(f"{sheet_name}情感平均分：{ave_score_l:.2f}")

In [ ]:
from snownlp import SnowNLP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# 可视化设置
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

def preprocess_data(df):
    """数据预处理函数"""
    processed = df.copy()
    
    # 处理评价星级
    processed['评价星级'] = processed['评价星级'].str.extract('star(\d)').astype(float)
    
    # 处理会员级别
    processed['是否PLUS会员'] = processed['级别'].apply(lambda x: 1 if 'PLUS' in str(x) else 0)
    
    # 处理追评时间（提取天数）
    processed['追评天数'] = processed['追评时间'].str.extract('购买(\d+)天后追评').fillna(0).astype(int)
    
    # 处理追评内容（是否有效）
    processed['有效追评'] = processed['追评内容'].apply(
        lambda x: 0 if pd.isnull(x) or '未填写' in str(x) else 1)
    
    # 计算情感得分
    processed['情感得分'] = processed['评价内容'].apply(
        lambda x: SnowNLP(x).sentiments if pd.notnull(x) else np.nan)
    
    # 情感分类
    processed['情感分类'] = processed['情感得分'].apply(
        lambda x: 0 if x < 0.4 else 1 if x < 0.6 else 2)
    
    # 商品属性编码
    le = LabelEncoder()
    processed['商品编码'] = le.fit_transform(processed['商品属性'])
    
    return processed.dropna()

def analyze_sheet(sheet_name, df):
    """单sheet分析函数"""
    print(f'\n正在分析: {sheet_name}')
    
    # ========== 可视化分析 ==========
    # 1. 星级分布
    plt.figure(figsize=(10, 5))
    sns.countplot(x='评价星级', data=df)
    plt.title(f'{sheet_name} - 评价星级分布')
    plt.show()

    # 2. 情感得分分布
    plt.figure(figsize=(10, 5))
    sns.histplot(df['情感得分'], bins=20, kde=True)
    plt.title(f'{sheet_name} - 情感得分分布')
    plt.show()

    # 3. 星级与情感得分关系
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='评价星级', y='情感得分', data=df)
    plt.title(f'{sheet_name} - 星级与情感得分关系')
    plt.show()

    # 4. PLUS会员情感分析
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='是否PLUS会员', y='情感得分', data=df)
    plt.title(f'{sheet_name} - PLUS会员情感分布')
    plt.xticks([0, 1], ['普通用户', 'PLUS会员'])
    plt.show()

    # ========== 相关性分析 ==========
    corr_matrix = df[['评价星级', '情感得分', '点赞数', '评论数', '追评天数', '是否PLUS会员']].corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title(f'{sheet_name} - 特征相关性分析')
    plt.show()

    # ========== 决策树建模 ==========
    # 准备特征
    features = df[['评价星级', '是否PLUS会员', '追评天数', '有效追评', '商品编码']]
    target = df['情感分类']
    
    # 训练决策树
    dt = DecisionTreeClassifier(max_depth=3)
    dt.fit(features, target)
    
    # 可视化决策树
    plt.figure(figsize=(20, 12))
    plot_tree(dt, 
             feature_names=features.columns,
             class_names=['负面', '中性', '正面'],
             filled=True,
             rounded=True)
    plt.title(f'{sheet_name} - 情感分类决策树')
    plt.show()

    # 输出分类报告
    print(f'\n{sheet_name} 分类报告：')
    print(classification_report(target, dt.predict(features)))

# 主程序
excel_file = pd.ExcelFile('京东评论.xlsx')

for sheet_name in excel_file.sheet_names:
    # 读取并预处理数据
    raw_df = excel_file.parse(sheet_name)
    processed_df = preprocess_data(raw_df)
    
    # 执行分析
    analyze_sheet(sheet_name, processed_df)

In [ ]:
# 词频统计，评论分析

import pandas as pd
import matplotlib.pyplot as plt
import jieba
import wordcloud as wc
from jieba import analyse #关键词提取模块
import numpy as np
from PIL import Image



# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取 Excel 文件
excel_file = pd.ExcelFile('京东好评差评.xlsx')

# 获取所有表名
sheet_names = excel_file.sheet_names

# 遍历每个 sheet
for sheet_name in sheet_names:
    print(f'正在分析表 {sheet_name}...')
    # 获取当前 sheet 的数据
    data_jingdong = excel_file.parse(sheet_name)
    bad_comment = data_jingdong[data_jingdong['评论类型'] == '差评']['评价内容']


    # 提取全部用户评价，并且组合在一个字符串evaluation当中
    evaluation = ''.join(bad_comment)
    jieba.load_userdict('用户自定义词典.txt')#加载用户自定义词典
    # print('使用自定义词典后分词结构如下：')
    words = jieba.lcut(evaluation) #lcut 会将分词结果保存在一个列表里
    #过滤掉停用词
    # 1.加载停用词列表
    with open('停用词.txt', encoding = 'utf-8') as f:
        stop_words = [w.strip() for w in f]
    # 2.从分词结果中过滤停用词      
    words = [w for w in words if w not in stop_words]
    # print(words)
    # words转化成pandas的系列类型
    words = pd.array(words)
    word_fre = words.value_counts()
    # 从evaluation提取关键词
    # 提取评分的最高的前70个词，要求只选择名词
    keyWords = analyse.textrank(evaluation,topK=70,allowPOS=['n'])
    keyWords = [word for word in keyWords if word in words]
    # print(keyWords)
    # 从word_fre中提取出所有keyWords的词频
    key_fre = word_fre[keyWords]
    # print(key_fre)
    # 可视化结果--画图
    #导入画图所需模块，词云图
    # 1.条形图
    plt.bar(key_fre.index[:10],key_fre.values[:10])
    plt.title(f'{sheet_name}评论关键词词频图')
    plt.show()
    # plt.savefig('联想ThinkPadT14p词频图.png')
    # 2.词云图
    # （1）转化为字典
    key_d = dict(zip(key_fre.index,key_fre.values))
    # print(key_d)
    # （2）画图
    myfont = r'C:\Windows\Fonts\simhei.ttf'
    mymask = np.array(Image.open(f'logo\y7000p.png'))
    fig=wc.WordCloud(font_path=myfont,background_color='white',mask=mymask).fit_words(key_d)
    plt.imshow(fig)
    plt.title(f'{sheet_name}词云图')
    plt.axis("off")
    plt.show()



